In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Presentación de la Solución

Responda las siguientes preguntas (en un cuaderno de ipython, documento de `.Rmd`, o `shinyapp` ):

1. Qué otros datos pueden ayudarlo a mejorar su predicción?
2. Cuál día de la semana presenta el número mayor de órdenes?
3. Cuál día de la semana presenta el menor número de órdenes?

Su solución debe llevarse a cabo en el repositorio `HW8_NombreApellido` y debe incluir:
- El código que detalla su modelo predictivo.
- Cualquier intuición y/o visualizaciones que usted obtenga a partir de esos datos.
- Las respuestas a las preguntas planteadas.